### Retrieval Scoring + ranking


In [43]:
## Init stuff
import os
import json
import psycopg2
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()  # reads .env if present

REDDIT_CLIENT_ID = os.getenv("REDDIT_CLIENT_ID")
REDDIT_CLIENT_SECRET = os.getenv("REDDIT_CLIENT_SECRET")
REDDIT_USER_AGENT = os.getenv("REDDIT_USER_AGENT") or "knowledge-collection-service/0.1"

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# --- connect to Postgres ---
conn = psycopg2.connect(
    host="localhost",
    port=5432,
    dbname="knowledgebase",
    user="stiw_user",
    password="stiw_pwd"
)
cursor = conn.cursor()
print("✅ Connected to Postgres and OpenAI")

✅ Connected to Postgres and OpenAI


Run once for Query & Query Embed, & test other systems for tweaks


In [44]:
user_input = input("Enter your query or goal: ").strip()
print(f"Query: --> {user_input}")

# Running HyDE style query improvement 

HyDE_prompt = f"""
You are an extremely empathetic Reddit user writing a raw, honest submission asking for help.
Write a detailed (150–250 word) Reddit post (title + body) from someone in exactly the same situation as the user below.
Make it the kind of thoughtful, non-ranty, "I know I need to change, please help me" post that gets tons of supportive, practical, life-changing advice in the comments.

Query / situation: {user_input}

Start directly with the post (include a realistic title and then the body). 
Do NOT write advice or a success story — only the help-seeking post.
"""

print("Creating a HyDE hypothetical Reddit submission (OP post)...")
HyDE_response = client.chat.completions.create(
    model="gpt-4o-mini",       
    messages=[
        {"role": "system", "content": "You are a Reddit user asking for help in a detailed, vulnerable way."},
        {"role": "user", "content": HyDE_prompt}
    ],
    temperature=0.8, # keeps it varied and natural
    max_tokens=500 # Wanna limit token usage here to create precise embeddings
)

HyDE_answer = HyDE_response.choices[0].message.content.strip()

print("\📄 HyDE generated hypothetical Reddit submission \n")
print(HyDE_answer)

# Embedd HyDE, query
q_emb = client.embeddings.create(
        model="text-embedding-3-small",
        input=HyDE_answer
    ).data[0].embedding
print("✅ Query Embedding generated...")

Query: --> Hi I have a bad relation ship with my parents, I suffer from depression and I am not fit at all how can I turn my life around
Creating a HyDE hypothetical Reddit submission (OP post)...
\📄 HyDE generated hypothetical Reddit submission 

**Title: Seeking Guidance to Heal My Relationship with My Parents and Improve My Life**

Hi everyone,

I’m reaching out because I feel completely lost and overwhelmed in my life. My relationship with my parents has been strained for years, and I don’t know how to mend it. We often argue, and I feel misunderstood and unappreciated. The constant tension at home makes everything worse, and I find myself feeling isolated and depressed. 

On top of that, I’ve struggled with my mental health for a while now. I often feel lethargic and unmotivated, which makes it hard to find the energy to engage in even the simplest tasks. I know I need to change my lifestyle; I’m not fit at all, and it’s taking a toll on my self-esteem. I feel like I’m stuck in th

In [45]:
print(q_emb)

[0.03392084687948227, 0.007214669603854418, 0.0016589955193921924, 0.07251445949077606, -0.01954559050500393, -0.02898847870528698, -0.027019858360290527, -0.003015126334503293, -0.0025608292780816555, 0.045516237616539, 0.001959156012162566, 0.011649474501609802, -0.00935094803571701, -0.04209819436073303, 0.017641868442296982, -0.002600039355456829, 0.029031744226813316, 0.03495923802256584, -0.019989069551229477, 0.07913421839475632, 0.05646263435482979, -0.002256612526252866, 0.03569476678967476, -0.016722457483410835, 0.027430888265371323, -0.025137770920991898, 0.04482397437095642, 0.041362665593624115, 0.017155122011899948, 0.009994534775614738, -0.03521883860230446, -0.004467254504561424, -0.0347861722111702, 0.01389932632446289, 0.006884763482958078, 0.0061492351815104485, -0.018182698637247086, -0.01059485599398613, 0.019156191498041153, -0.025873297825455666, 0.007322835735976696, 0.002348553389310837, 0.03257958963513374, 0.03201712667942047, -0.020800314843654633, -0.01314

Running through retrieved chunks for new scoring


In [50]:
print("Retrieveing top matching submissions....")
cursor.execute("""
            SELECT submission_id, domain_id, title, selftext, permalink
            FROM submissions
            ORDER BY embedding <=> %s::vector
            LIMIT 15;
        """, (json.dumps(q_emb),))
top_subs = cursor.fetchall()

if not top_subs:
    print("❌ No Relevant submissions found")
else:
    print(f"✅ Found {len(top_subs)} relevant submissions")

# Printing submissions
print(json.dumps(top_subs, indent=5))

Retrieveing top matching submissions....
✅ Found 15 relevant submissions
[
     [
          "ps609e",
          "6ba11411-ff82-4528-89f4-b3193ce7189f",
          "I don\u2019t want to be my fathers caregiver, I just want to die.",
          "Hi, I don\u2019t know exactly what I\u2019m looking for here, but if you happen to stumble upon my venting session and read it, thank you for seeing me. \n\nMy parents probably should have divorced when my brother and I were kids, but they didn\u2019t, and becasue of his career in offshore drilling, my dad has never lived in the same city as me for the entirety of my life. 4 years ago he came home after engineering jobs in the oilfield started declining, and unbeknownst to any of the family, he was very ill. My mother moved to Florida immediately when he moved in, and my brother lived out of town about an hour and a half away. Long story short, my brother found my dad in a grave state, called me, and split to go back to his life. I saved my dad in 

In [51]:
# Collect related comments
context_blocks = []
sources = []  # track all permalinks for final output

top_submission_ids = [row[0] for row in top_subs]
print(top_submission_ids)


# Pulling in the most Relevant comments from only the top posts that we get
# There is also a small scoring boost for the high upvoted comments, at the time of querying
# Relevance first tho, only then popularity
query = """
WITH ranked_comments AS (
    SELECT 
        c.body,
        c.score,
        s.title,
        c.submission_id,
        s.permalink,
        c.embedding <=> %s::vector AS distance
    FROM comments c
    JOIN submissions s ON c.submission_id = s.submission_id
    WHERE c.submission_id = ANY(%s)
)
SELECT 
    body,
    score,
    title,
    submission_id,
    permalink,
    distance
FROM ranked_comments
ORDER BY 
    distance + (1.0 / (score + 10)) ASC 
LIMIT 20;
"""

# Note the order of parameters is now --> embedding, submission_ids list
cursor.execute(query, (json.dumps(q_emb), top_submission_ids))
relevant_comments = cursor.fetchall()

print(f"\n✅ Retrieved {len(relevant_comments)} relevant comments\n")

# for i, (body, score, title, permalink, distance) in enumerate(relevant_comments, 1):
#     print(f"{i:2d}. ↑{score}  (dist: {distance:.4f}) | {title}")
#     print(f"     {body.strip()[:900]}{'...' if len(body) > 900 else ''}")
#     print(f"     https://reddit.com{permalink}\n")
# print("="*120)

# Context map contains : submission_id -> {submission_info, comments}
context_map = {}

# building submission details
submission_details = {}
for sub in top_subs:
    submission_id = sub[0]
    title = sub[2]
    selftext = sub[3]
    permalink = sub[4]
    
    submission_details[submission_id] = {
        'title': title,
        'selftext': selftext,
        'permalink': permalink
    }


for comment in relevant_comments:
    body = comment[0]
    score = comment[1]
    title = comment[2]
    submission_id = comment[3]
    permalink = comment[4]
    distance = comment[5]
    
    if submission_id not in context_map:
        # Get selftext from submission_details
        sub_info = submission_details.get(submission_id, {})
        
        context_map[submission_id] = {
            'title': title,
            'permalink': permalink,
            'selftext': sub_info.get('selftext', ''),
            'comments': []
        }
    
    context_map[submission_id]['comments'].append({
        'body': body,
        'score': score,
        'distance': distance
    })

print(f"✅ Built context map with {len(context_map)} submissions\n")
print(json.dumps(context_map, indent=5))

['ps609e', 'ca66sv', '1ky96h6', '1m0jckk', 'a2dmje', 'gqe0j8', 'lw0bsm', '121rnry', '1jvm5jj', 'czhvrp', '1fitbc5', 'juou3v', '1lqf3d0', '1f9b0hh', '1kc67ha']

✅ Retrieved 20 relevant comments

✅ Built context map with 9 submissions

{
     "lw0bsm": {
          "title": "[Advice] In my mid-20s I was lost, socially awkward, and had zero discipline. But in just a few years, my life turned completely around ---> Here's my advice to 20-somethings thirsting for a change",
          "permalink": "https://www.reddit.com/r/getdisciplined/comments/lw0bsm/advice_in_my_mid20s_i_was_lost_socially_awkward/",
          "selftext": "Starting in my mid-20s, my life completely transformed. In just a few years' time.\n\n**I am writing this post to share the lessons I've learned. Including practical tips on how to change your life in your 20s.**\n\nI'll be dividing things into 3 sections:\n\n1. My story of transformation (in brief)\n2. What NOT to do if you want to improve your life (things to avoid)\n3

# Looking at more fetched comment details


In [52]:
# Prepare for generation, 
context_blocks = []
sources = []

for submission_id, data in context_map.items():
    title = data['title']
    permalink = data['permalink']
    selftext = data['selftext']
    comments = data['comments']
    
    # Format comments
    joined_comments = "\n".join([
        f"- (↑{c['score']}) {c['body'][:600]}..." 
        for c in comments
    ])
    
    # Build block
    block = f"""
[POST] "{title}"
URL: {permalink}

{selftext[:1200] if selftext else "(No text body)"}

Top Comments:
{joined_comments}
"""
    context_blocks.append(block)
    sources.append(f"- {title}: {permalink}")

# Final context text to pass to LLM
context_text = "\n\n---\n\n".join(context_blocks)

print("CONTEXT FOR LLM:")
print(context_text)
print("\n" )
print("SOURCES:")
print("\n".join(sources))

CONTEXT FOR LLM:

[POST] "[Advice] In my mid-20s I was lost, socially awkward, and had zero discipline. But in just a few years, my life turned completely around ---> Here's my advice to 20-somethings thirsting for a change"
URL: https://www.reddit.com/r/getdisciplined/comments/lw0bsm/advice_in_my_mid20s_i_was_lost_socially_awkward/

Starting in my mid-20s, my life completely transformed. In just a few years' time.

**I am writing this post to share the lessons I've learned. Including practical tips on how to change your life in your 20s.**

I'll be dividing things into 3 sections:

1. My story of transformation (in brief)
2. What NOT to do if you want to improve your life (things to avoid)
3. What to DO to change your life

**If you prefer video format, check out the first post in my profile.**

## Section 1: My Story (In Brief)

**My life dramatically changed from my mid to late 20s. In my mid-20s I:**

* Was socially awkward
* Rarely exercised
* Ate unhealthy foods
* Had no idea wha

In [53]:
print("Generating output...")
prompt = f"""
    You are an empathetic assistant that uses examples from real Reddit users' discussions
    to guide someone based on their current life goal.

    User Query:
    {user_input}

    Here are similar cases from Reddit:
    {context_text}

    Using the experiences and insights quoted above, write an actionable plan
    to help the user.
    At the END of your response, include a "Sources" section listing all the Reddit discussions you referenced.
    Format each source as: "- [Post Title]: [full permalink URL]"
"""

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful, grounded life-strategy coach."},
        {"role": "user", "content": prompt}
    ],
    temperature=0.7
)

print("\n--- LLM RESPONSE ---\n")
print(response.choices[0].message.content)

Generating output...

--- LLM RESPONSE ---

It's commendable that you're seeking to turn your life around despite facing challenges with your parents, depression, and fitness. Here’s a structured plan based on various experiences shared by Reddit users who have faced similar struggles:

### Actionable Plan for Transformation

#### 1. **Understanding Your Situation**
   - **Acknowledge Your Feelings**: Like the user who shared that they felt lost and socially awkward in their mid-20s, it's important to recognize and validate your feelings of sadness and frustration. Understand that you're not alone; many people have been in similar situations and have successfully turned their lives around.
   - **Reflect on Your Relationship with Your Parents**: Consider what specific aspects of your relationship with your parents are causing distress. Is it communication, support, or understanding? Engaging in open dialogue, if possible, can be beneficial.

#### 2. **Mental Health Focus**
   - **Seek 